In [1]:
from simtk.openmm import app
import simtk.openmm as mm
from simtk import unit
pdb = app.PDBFile('butane.pdb')
forcefield = app.ForceField('butane.gaff2.xml')
system = forcefield.createSystem(pdb.topology, nonbondedMethod=app.NoCutoff, constraints=app.HBonds)
integrator = mm.LangevinIntegrator(298.15*unit.kelvin, 5.0/unit.picoseconds, 2.0*unit.femtoseconds)
integrator.setConstraintTolerance(1e-5)
platform = mm.Platform.getPlatformByName('Reference')
simulation = app.Simulation(pdb.topology, system, integrator, platform)
simulation.context.setPositions(pdb.positions)

In [2]:
print('Minimizing...')

st = simulation.context.getState(getPositions=True,getEnergy=True)
print(F"Potential energy before minimization is {st.getPotentialEnergy()}")

simulation.minimizeEnergy(maxIterations=100)

st = simulation.context.getState(getPositions=True,getEnergy=True)
print(F"Potential energy after minimization is {st.getPotentialEnergy()}")

Minimizing...
Potential energy before minimization is 5.797528654036221 kJ/mol
Potential energy after minimization is 5.295784444937283 kJ/mol


In [3]:
from sys import stdout

print('Equilibrating...')

simulation.reporters.append(app.StateDataReporter(stdout, 250, step=True, 
    potentialEnergy=True, temperature=True, separator=','))
simulation.context.setVelocitiesToTemperature(150.0*unit.kelvin)
simulation.step(5000)

Equilibrating...
#"Step","Potential Energy (kJ/mole)","Temperature (K)"
250,25.681036534494396,326.70336343339477
500,40.004743766057054,250.10974268198007
750,37.38463557284553,428.9841107507136
1000,66.15505505856767,291.4784892493697
1250,30.968409904156758,414.75074031358173
1500,30.527989446793548,233.8808469300103
1750,33.9554694304348,296.04630351747863
2000,46.376735790756655,251.05419280284409
2250,46.96566968699414,210.61939361493108
2500,44.11659132760187,231.97879632406935
2750,41.15239291232457,414.61784081038473
3000,43.36132979376796,256.5030351190406
3250,32.51229160581751,476.64224956779543
3500,63.85785221384515,230.22943829361466
3750,45.834562219572675,298.6900667493698
4000,76.32010872180426,359.11783281787444
4250,32.07853153235199,223.107398009778
4500,34.568497182802226,348.2990386062612
4750,45.853327884796464,188.69415089159335
5000,45.95950697618548,308.9148992467615


In [4]:
import time as time

print('Running Production')

# begin timer
tinit=time.time()

# clear simulation reporters
simulation.reporters.clear()

# reinitialize simulation reporters. Done because we want different information printed from the production run
# than the equilibriation run. 
# output basic simulation information below every 500,000 steps - 2 x 500,000 = 1,000,000fs or 1ns
simulation.reporters.append(app.StateDataReporter(stdout, 500000, step=True, time=True, potentialEnergy=True,
                            temperature=True, speed=True, separator=','))

# write traj to a DCD file every 100 steps - 0.2ps
simulation.reporters.append(app.DCDReporter('butane_sim.dcd', 100))

# run simulation for 2.0x10^7 steps - 40ns
simulation.step(20000000)

# end timer
tfinal=time.time()
print('Done!')
print('Time required for simulation:', tfinal-tinit, 'seconds')

Running Production
#"Step","Time (ps)","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
500000,999.9999999901769,34.30501752522847,158.56774693281582,0
1000000,1999.9999999665301,30.7493449452922,416.6774939564473,1.63e+04
1500000,2999.9999999428833,32.42238763320715,302.34176854445604,1.63e+04
2000000,3999.9999999192364,41.31171117729838,256.2662645007854,1.62e+04
2500000,5000.000000101135,34.26054472640648,380.8029739067473,1.61e+04
3000000,6000.000000304862,49.00276841622885,366.63684079383313,1.61e+04
3500000,7000.0000005085885,42.960783108855736,291.5114076369063,1.61e+04
4000000,8000.000000712315,44.922505802412395,151.70825889183178,1.62e+04
4500000,9000.000000916041,31.523159591575624,179.02583793068973,1.62e+04
5000000,10000.000001119768,46.45523599582569,311.0229204208048,1.62e+04
5500000,11000.000001323495,44.51616724620192,228.07175637963599,1.62e+04
6000000,12000.000001527222,51.24809284531176,294.75441150702096,1.62e+04
6500000,13000.000001730948,29.6908021